# Đồ án 1 - Color Compression <a class="anchor" id="c2"></a>

Họ tên: Dương Chí Thông

MSSV: 20127634

Lớp: 20CLC08

## Cài đặt thuật toán

### Import các thư viện cần thiết

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

### Khởi tạo centroids

In [ ]:
def init_centroids_kmean(img_1d,k_clusters,init_centroids):
    if init_centroids == 'random':
        return np.random.randint(0,255, size=(k_clusters, len(img_1d[0])))
    elif init_centroids == 'in_pixels':
        return img_1d[np.random.choice(img_1d.shape[0], size=k_clusters, replace=False)]
    else:
        raise ValueError('init_centroids must be either "random" or "in_pixels"')

### Xác định các điểm gần trung tâm

In [ ]:
def get_label(img_1d, centroid):
    list_distance=[]
    for i in range(0,len(centroid)):
        distance = np.linalg.norm(np.subtract(img_1d ,centroid[i]), axis=1) #tính khoảng cách giữa ảnh và từng centroid
        list_distance.append(distance) #lưu khoảng cách vào mảng
    label = np.argmin(list_distance, axis=0) #lấy index của khoảng cách nhỏ nhất
    return label

### Cập nhật lại trung tâm

In [ ]:
def get_centroid(img_1d, label, k_clusters):
    centroid=np.zeros((k_clusters, len(img_1d[0])))
    for i in range (k_clusters):
        k=img_1d[label==i]
        if(len(k)!=0):
            centroid[i]=np.mean(k,axis=0)
    return centroid 

### Thuật toán Kmeans

In [ ]:
def kmeans(img_1d, k_clusters, max_iter, init_centroids='random'):
    flag=0
    centroid=init_centroids_kmean(img_1d,k_clusters,init_centroids)
    for i in range(max_iter):
        label = get_label(img_1d, centroid) #lấy label của ảnh
        new_centroid = get_centroid(img_1d, label, k_clusters) #cập nhật centroid
        if np.array_equal(centroid,new_centroid): # neu khong co su thay đổi về centroid thì dừng vòng lặp
            flag=1
            break
        centroid = new_centroid #cập nhật centroid mới
    if(flag==0): # nếu chạy hết vòng lăp thì ta tiến cập nhật lại label
        label=get_label(img_1d,centroid)
    return label, centroid

### Hàm main

In [ ]:
if __name__ == '__main__':
    img_file=input("Enter the image file name: ")
    img = Image.open(img_file)
    img_1d = np.array(img)
    img_shape=img_1d.shape
    img_1d = img_1d.reshape((img_1d.shape[0] * img_1d.shape[1], img_1d.shape[2]))
    k_clusters = int(input("Enter the number of clusters: "))
    max_iter = int(input("Enter the maximum number of iterations: "))
    init_centroids = input("Enter the initial centroids: ")
    print("K-means clustering...")
    label, centroid = kmeans(img_1d,k_clusters,max_iter, "random")
    label = label.reshape(img.size[0], img.size[1]) #reshape label về ảnh
    result = centroid[label].astype(np.uint8) #thay thế điểm ảnh bằng centroid của nó
    result = result.reshape(img_shape) #reshape result về ảnh
    plt.imshow(result) #hiển thị ảnh
    plt.show()
    file_name=input("Enter the file name want to save [filename.(jpg/png/pdf)]: ")
    plt.imsave(file_name,result)